In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/CV

/content/drive/.shortcut-targets-by-id/1wu7ijWhY1_HJddIenGyM7F6fvOStsSds/CV


In [2]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import numpy as np
import shutil
from shutil import copyfile

# Xử lý dữ liệu

In [ ]:
data_path = 'Data'
os.listdir(data_path)

['Annotations',
 'sample-nightClip1',
 'nightSequence2',
 'nightSequence1',
 'sample-dayClip6',
 'daySequence1',
 'dayTrain',
 'nightTrain',
 'daySequence2']

In [ ]:
day_train_path = 'Data/Annotations/Annotations/dayTrain/'
night_train_path = 'Data/Annotations/Annotations/nightTrain/'

In [ ]:
train_day = []
for clipName in tqdm(sorted(os.listdir(day_train_path))):
    if 'dayClip' not in clipName:
        continue
    df = pd.read_csv(os.path.join(day_train_path,clipName,'frameAnnotationsBOX.csv'),sep=';')
    train_day.append(df)

train_day_df = pd.concat(train_day,axis=0)
train_day_df['isNight'] = 0

100%|██████████| 13/13 [00:10<00:00,  1.19it/s]


In [ ]:
train_night = []
for clipName in tqdm(sorted(os.listdir(night_train_path))):
    if 'nightClip' not in clipName:
        continue
    df = pd.read_csv(os.path.join(night_train_path,clipName,'frameAnnotationsBOX.csv'),sep=';')
    train_night.append(df)

train_night_df = pd.concat(train_night,axis=0)
train_night_df['isNight'] = 1

100%|██████████| 5/5 [00:04<00:00,  1.12it/s]


In [ ]:
df = pd.concat([train_day_df,train_night_df],axis=0)

df.head()

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin file,Origin frame number,Origin track,Origin track frame number,isNight
0,dayTraining/dayClip1--00000.jpg,go,698,333,710,358,dayTraining/dayClip1/dayClip1Shutter0.000800-G...,0,dayTraining/dayClip1/dayClip1Shutter0.000800-G...,0,0
1,dayTraining/dayClip1--00000.jpg,go,846,391,858,411,dayTraining/dayClip1/dayClip1Shutter0.000800-G...,0,dayTraining/dayClip1/dayClip1Shutter0.000800-G...,0,0
2,dayTraining/dayClip1--00001.jpg,go,698,337,710,357,dayTraining/dayClip1/dayClip1Shutter0.000800-G...,1,dayTraining/dayClip1/dayClip1Shutter0.000800-G...,1,0
3,dayTraining/dayClip1--00001.jpg,go,847,390,859,410,dayTraining/dayClip1/dayClip1Shutter0.000800-G...,1,dayTraining/dayClip1/dayClip1Shutter0.000800-G...,1,0
4,dayTraining/dayClip1--00002.jpg,go,698,331,710,356,dayTraining/dayClip1/dayClip1Shutter0.000800-G...,2,dayTraining/dayClip1/dayClip1Shutter0.000800-G...,2,0


In [ ]:
df = df.drop(['Origin file','Origin track','Origin track frame number'],axis=1)

In [ ]:
def changeFilename(x):
    filename = x.Filename
    isNight = x.isNight
    
    splitted = filename.split('/')
    clipName = splitted[-1].split('--')[0]
    if isNight:
        return os.path.join(data_path,f'nightTrain/nightTrain/{clipName}/frames/{splitted[-1]}')
    else:
        return os.path.join(data_path,f'dayTrain/dayTrain/{clipName}/frames/{splitted[-1]}')

df['Filename'] = df.apply(changeFilename,axis=1)
df.columns = ['image_id','label','x_min','y_min','x_max','y_max','frame','isNight']

df

,image_id,label,x_min,y_min,x_max,y_max,frame,isNight
0,Data/dayTrain/dayTrain/dayClip1/frames/dayClip...,go,698,333,710,358,0,0
1,Data/dayTrain/dayTrain/dayClip1/frames/dayClip...,go,846,391,858,411,0,0
2,Data/dayTrain/dayTrain/dayClip1/frames/dayClip...,go,698,337,710,357,1,0
3,Data/dayTrain/dayTrain/dayClip1/frames/dayClip...,go,847,390,859,410,1,0
4,Data/dayTrain/dayTrain/dayClip1/frames/dayClip...,go,698,331,710,356,2,0
...,...,...,...,...,...,...,...,...
3231,Data/nightTrain/nightTrain/nightClip5/frames/n...,go,937,221,976,290,1450,1
3232,Data/nightTrain/nightTrain/nightClip5/frames/n...,go,467,36,512,117,1451,1
3233,Data/nightTrain/nightTrain/nightClip5/frames/n...,go,937,219,976,288,1451,1
3234,Data/nightTrain/nightTrain/nightClip5/frames/n...,go,460,33,506,114,1452,1


In [ ]:
label_to_idx = {'go':0, 'warning':1, 'stop': 2}
idx_to_label = {v:k for k,v in label_to_idx.items()}

def changeAnnotation(x):
    if 'go' in x['label']:
        return label_to_idx['go']
    elif 'warning' in x['label']:
        return label_to_idx['warning']
    elif 'stop' in x['label']:
        return label_to_idx['stop']
    
df['label'] = df.apply(changeAnnotation,axis=1)

annotation_tags = df['label'].unique()

In [ ]:
df['clipNames'] = df[['image_id']].applymap(lambda x: x.split('/')[5])
df['clipNames'].unique()

array(['dayClip1--00000.jpg', 'dayClip1--00001.jpg',
       'dayClip1--00002.jpg', ..., 'nightClip5--01450.jpg',
       'nightClip5--01451.jpg', 'nightClip5--01452.jpg'], dtype=object)

In [ ]:
df.drop(['image_id','frame','isNight'],axis=1)

,label,x_min,y_min,x_max,y_max,clipNames
0,0,698,333,710,358,dayClip1--00000.jpg
1,0,846,391,858,411,dayClip1--00000.jpg
2,0,698,337,710,357,dayClip1--00001.jpg
3,0,847,390,859,410,dayClip1--00001.jpg
4,0,698,331,710,356,dayClip1--00002.jpg
...,...,...,...,...,...,...
3231,0,937,221,976,290,nightClip5--01450.jpg
3232,0,467,36,512,117,nightClip5--01451.jpg
3233,0,937,219,976,288,nightClip5--01451.jpg
3234,0,460,33,506,114,nightClip5--01452.jpg


In [ ]:
image = Image.open('Data/nightTrain/nightTrain/nightClip1/frames/nightClip1--00001.jpg')
width, height = image.size
width, height

(1280, 960)

In [ ]:
def save_to_txt(dataFrame):
  for index, row in df.iterrows():
    label = row['label']
    x_min = row['x_min']
    x_max = row['x_max']
    y_min = row['y_min']
    y_max = row['y_max']
    file_name = os.path.splitext(row['clipNames'])[0]

    x_center = ((x_max + x_min) / 2) / 1280
    y_center = ((y_max + y_min) / 2) / 960
    width = (x_max - x_min) / 1280
    height = (y_max - y_min) / 960

    with open(f'{file_name}.txt', 'a') as txt_file:
        txt_file.write(f'{label} {x_center} {y_center} {width} {height}\n')

In [ ]:
%cd /content/drive/MyDrive/CV/Data_YOLO/label

/content/drive/.shortcut-targets-by-id/1wu7ijWhY1_HJddIenGyM7F6fvOStsSds/CV/Data_YOLO/label


In [ ]:
save_to_txt(df)

In [ ]:
def merge_imgs(origin_path, target_path):
  files = os.listdir(origin_path)
  for file_name in files:
   shutil.copy(origin_path + file_name, target_path + file_name)

In [ ]:
target_path = f'/content/drive/MyDrive/CV/Data_YOLO/images/'

for i in range(1,14):
  origin_path = f'/content/drive/MyDrive/CV/Data/dayTrain/dayTrain/dayClip{i}/frames/'
  merge_imgs(origin_path, target_path)

for i in range(1,6):
  origin_path = f'/content/drive/MyDrive/CV/Data/nightTrain/nightTrain/nightClip{i}/frames/'
  merge_imgs(origin_path, target_path)

In [ ]:
image_folder = '/content/drive/MyDrive/CV/Data_YOLO/images/' 
txt_folder = '/content/drive/MyDrive/CV/Data_YOLO/label/'  

for txt_file in os.listdir(txt_folder):
    txt_file_path = os.path.join(txt_folder, txt_file)
    image_file = os.path.splitext(txt_file)[0] + '.jpg'

    image_file_path = os.path.join(image_folder, image_file)

    if os.path.isfile(image_file_path):
      
        destination_path = os.path.join('/content/drive/MyDrive/CV/Data_YOLO/image/', image_file)
        shutil.copy(image_file_path, destination_path)

In [ ]:
list_label = os.listdir('/content/drive/MyDrive/CV/Data_YOLO/label')
len(list_label)

18013

In [ ]:
list_img = os.listdir('/content/drive/MyDrive/CV/Data_YOLO/image')
len(list_img)

8191

# Split train, val, test

In [ ]:
import random

def split_txt_folder(input_folder, output_train, output_test, output_valid, split_ratio):
    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)

        choice = random.choices(['train', 'test', 'valid'], split_ratio)[0]
        if choice == 'train':
            destination_folder = output_train
        elif choice == 'test':
            destination_folder = output_test
        else:
            destination_folder = output_valid

        destination_path = os.path.join(destination_folder, file_name)
        shutil.copyfile(file_path, destination_path)

folder = '/content/drive/MyDrive/CV/Data_YOLO/image/'

train = '/content/drive/MyDrive/CV/Data_YOLO/train/images/'
test = '/content/drive/MyDrive/CV/Data_YOLO/test/images/'
valid = '/content/drive/MyDrive/CV/Data_YOLO/valid/images/'

split_ratio = [0.7, 0.1, 0.2]

split_txt_folder(folder, train, test, valid, split_ratio)

In [ ]:
list_images = os.listdir('/content/drive/MyDrive/CV/Data_YOLO/train/images')
len(list_images)

5715

In [ ]:
list_images = os.listdir('/content/drive/MyDrive/CV/Data_YOLO/test/images')
len(list_images)

820

In [ ]:
list_images = os.listdir('/content/drive/MyDrive/CV/Data_YOLO/valid/images')
len(list_images)

1656

In [ ]:
image_folder = '/content/drive/MyDrive/CV/Data_YOLO/train/images/' 
txt_folder = '/content/drive/MyDrive/CV/Data_YOLO/label/'  

for image_file in os.listdir(image_folder):
    image_file_path = os.path.join(image_folder, image_file)
    txt_file = os.path.splitext(image_file)[0] + '.txt'

    txt_file_path = os.path.join(txt_folder, txt_file)

    if os.path.isfile(txt_file_path):
      
        destination_path = os.path.join('/content/drive/MyDrive/CV/Data_YOLO/train/labels/', txt_file)
        shutil.copy(txt_file_path, destination_path)

In [ ]:
list_label = os.listdir('/content/drive/MyDrive/CV/Data_YOLO/train/labels/')
len(list_label)

5715

In [ ]:
image_folder = '/content/drive/MyDrive/CV/Data_YOLO/test/images/' 
txt_folder = '/content/drive/MyDrive/CV/Data_YOLO/label/'  

for image_file in os.listdir(image_folder):
    image_file_path = os.path.join(image_folder, image_file)
    txt_file = os.path.splitext(image_file)[0] + '.txt'

    txt_file_path = os.path.join(txt_folder, txt_file)

    if os.path.isfile(txt_file_path):
      
        destination_path = os.path.join('/content/drive/MyDrive/CV/Data_YOLO/test/labels/', txt_file)
        shutil.copy(txt_file_path, destination_path)

In [ ]:
list_label = os.listdir('/content/drive/MyDrive/CV/Data_YOLO/test/labels/')
len(list_label)

820

In [ ]:
image_folder = '/content/drive/MyDrive/CV/Data_YOLO/valid/images/' 
txt_folder = '/content/drive/MyDrive/CV/Data_YOLO/label/'  

for image_file in os.listdir(image_folder):
    image_file_path = os.path.join(image_folder, image_file)
    txt_file = os.path.splitext(image_file)[0] + '.txt'

    txt_file_path = os.path.join(txt_folder, txt_file)

    if os.path.isfile(txt_file_path):
      
        destination_path = os.path.join('/content/drive/MyDrive/CV/Data_YOLO/valid/labels/', txt_file)
        shutil.copy(txt_file_path, destination_path)

In [ ]:
list_label = os.listdir('/content/drive/MyDrive/CV/Data_YOLO/valid/labels/')
len(list_label)

1656

# Create yaml file


In [ ]:
data_yaml = f"""
    train: /content/drive/MyDrive/CV/Data_YOLO/train
    val: /content/drive/MyDrive/CV/Data_YOLO/valid
    test: /content/drive/MyDrive/CV/Data_YOLO/test
    nc: 3
    names:
        0 : go
        1 : warning
        2 : stop
    """

In [ ]:
with open('/content/drive/MyDrive/CV/Data_YOLO/data.yaml', 'a+') as f:
    f.write(data_yaml)

# Train

In [3]:
%cd /content/drive/MyDrive/CV/yolov8

/content/drive/.shortcut-targets-by-id/1wu7ijWhY1_HJddIenGyM7F6fvOStsSds/CV/yolov8


In [4]:
!pip install ultralytics

import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics YOLOv8.0.115 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.3/78.2 GB disk)


In [ ]:
!yolo detect train data="/content/drive/MyDrive/CV/Data_YOLO/data.yaml" model=yolov8x.pt epochs=100 imgsz=640

100% 131M/131M [00:00<00:00, 184MB/s]
Ultralytics YOLOv8.0.114 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/content/drive/MyDrive/CV/Data_YOLO/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=to

In [ ]:
!yolo detect train resume data="/content/drive/MyDrive/CV/Data_YOLO/data.yaml" model='runs/detect/train/weights/last.pt' epochs=100 imgsz=640

Ultralytics YOLOv8.0.114 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=runs/detect/train/weights/last.pt, data=/content/drive/MyDrive/CV/Data_YOLO/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, kera

In [ ]:
!yolo predict model='runs/detect/train/weights/best.pt' source='/content/drive/MyDrive/CV/test.jpg' imgsz=640

Ultralytics YOLOv8.0.114 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68126457 parameters, 0 gradients

image 1/1 /content/drive/MyDrive/CV/test.jpg: 480x640 3 stops, 99.3ms
Speed: 6.1ms preprocess, 99.3ms inference, 283.3ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict


In [ ]:
!yolo predict model='runs/detect/train/weights/best.pt' source='/content/drive/MyDrive/CV/test.mp4' 

Ultralytics YOLOv8.0.114 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68126457 parameters, 0 gradients

video 1/1 (1/3221) /content/drive/MyDrive/CV/test.mp4: 384x640 5 gos, 65.5ms
video 1/1 (2/3221) /content/drive/MyDrive/CV/test.mp4: 384x640 5 gos, 55.5ms
video 1/1 (3/3221) /content/drive/MyDrive/CV/test.mp4: 384x640 4 gos, 55.8ms
video 1/1 (4/3221) /content/drive/MyDrive/CV/test.mp4: 384x640 3 gos, 41.8ms
video 1/1 (5/3221) /content/drive/MyDrive/CV/test.mp4: 384x640 4 gos, 41.8ms
video 1/1 (6/3221) /content/drive/MyDrive/CV/test.mp4: 384x640 4 gos, 41.8ms
video 1/1 (7/3221) /content/drive/MyDrive/CV/test.mp4: 384x640 3 gos, 40.0ms
video 1/1 (8/3221) /content/drive/MyDrive/CV/test.mp4: 384x640 4 gos, 32.5ms
video 1/1 (9/3221) /content/drive/MyDrive/CV/test.mp4: 384x640 4 gos, 31.3ms
video 1/1 (10/3221) /content/drive/MyDrive/CV/test.mp4: 384x640 3 gos, 31.2ms
video 1/1 (11/3221) /content/drive/MyDrive/CV/test.mp4: 384x640 3 gos, 3

In [5]:
!yolo predict model='runs/detect/train/weights/best.pt' source='/content/drive/MyDrive/CV/test2.jpg' imgsz=640

Ultralytics YOLOv8.0.115 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68126457 parameters, 0 gradients

image 1/1 /content/drive/MyDrive/CV/test2.jpg: 416x640 (no detections), 63.6ms
Speed: 11.6ms preprocess, 63.6ms inference, 74.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict3


In [6]:
!yolo predict model='runs/detect/train/weights/best.pt' source='/content/drive/MyDrive/CV/test3.jpg' imgsz=640

Ultralytics YOLOv8.0.115  Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68126457 parameters, 0 gradients

image 1/1 /content/drive/MyDrive/CV/test3.jpg: 640x480 1 go, 71.2ms
Speed: 3.3ms preprocess, 71.2ms inference, 194.3ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict4
